In [1]:
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python
!pip install -q opencv-python==4.8.1.78
!pip install -q ultralytics torch torchvision transformers
!pip install -q gdown

Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 30.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━

## download data

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1kuhTWOjlNFURhCIAf6lHSIqh6l1kTHA0/view?usp=drive_link # train
!gdown --fuzzy https://drive.google.com/file/d/19AzlsLF2u7IG23ynw6RrMg2F9jd76xNR/view?usp=drive_link # public test

Downloading...
From (original): https://drive.google.com/uc?id=1kuhTWOjlNFURhCIAf6lHSIqh6l1kTHA0
From (redirected): https://drive.google.com/uc?id=1kuhTWOjlNFURhCIAf6lHSIqh6l1kTHA0&confirm=t&uuid=9a0c3cd2-26be-42e9-ae55-0a9104d0e75b
To: /kaggle/working/train.zip
100%|████████████████████████████████████████| 636M/636M [00:06<00:00, 99.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=19AzlsLF2u7IG23ynw6RrMg2F9jd76xNR
From (redirected): https://drive.google.com/uc?id=19AzlsLF2u7IG23ynw6RrMg2F9jd76xNR&confirm=t&uuid=2b014a75-dba6-432b-ad71-cada0faa013b
To: /kaggle/working/public_test.zip
100%|█████████████████████████████████████████| 281M/281M [00:02<00:00, 107MB/s]


In [3]:
%%capture
!unzip train.zip
!unzip public_test.zip

## setting

In [4]:
import os
import json
import cv2
import yaml
import random
import torch
import shutil
from pathlib import Path
import numpy as np
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import glob
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Config
DATASET_ROOT = "train"
TEST_ROOT = "public_test/samples"
ANNOTATIONS_PATH = os.path.join(DATASET_ROOT, "annotations/annotations.json")
SAMPLES_DIR = os.path.join(DATASET_ROOT, "samples")
WORK_DIR = "/kaggle/working/stream_dataset"
os.makedirs(WORK_DIR, exist_ok=True)
OUTPUT_PATH = "/kaggle/working/submission.json"

TRAIN_RATIO = 0.85
EPOCHS = 50
BATCH = 32
NUM_WORKERS = 4
YOLO_MODEL = "yolov8l.pt"
CLASS_NAMES = ["target"]

In [6]:
# model = YOLO(TRAINED_MODEL_PATH)

# model.info()

In [7]:
# Load annotations
with open(ANNOTATIONS_PATH, "r") as f:
    annotations = json.load(f)

video_ids = [a["video_id"] for a in annotations]
ann_dict = {a["video_id"]: a for a in annotations}

In [8]:
# Split train/val
random.seed(42)
random.shuffle(video_ids)
split_idx = int(len(video_ids) * TRAIN_RATIO)
train_videos = video_ids[:split_idx]
val_videos = video_ids[split_idx:]
print(f"Loaded {len(video_ids)} videos — Train: {len(train_videos)}, Val: {len(val_videos)}")

Loaded 14 videos — Train: 11, Val: 3


In [9]:
# Prepare directories
def prepare_dirs(base_dir):
    imgs = os.path.join(base_dir, "images")
    lbls = os.path.join(base_dir, "labels")
    os.makedirs(imgs, exist_ok=True)
    os.makedirs(lbls, exist_ok=True)
    return imgs, lbls

train_img_dir, train_lbl_dir = prepare_dirs(os.path.join(WORK_DIR, "train"))
val_img_dir, val_lbl_dir = prepare_dirs(os.path.join(WORK_DIR, "val"))

In [10]:
EMPTY_FRAME_STEP = 30 

def convert_to_yolo_format(x1, y1, x2, y2, img_width, img_height, class_id):
    dw = 1.0 / img_width
    dh = 1.0 / img_height
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0
    width = x2 - x1
    height = y2 - y1
    
    x_center = max(0, min(1, x_center * dw))
    y_center = max(0, min(1, y_center * dh))
    width = max(0, min(1, width * dw))
    height = max(0, min(1, height * dh))
    
    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

def extract_frames_and_labels(video_id, mode="train"):
    video_dir = os.path.join(SAMPLES_DIR, video_id)
    video_path = os.path.join(video_dir, "drone_video.mp4")
    
    if not os.path.exists(video_path):
        return f"Missing video: {video_path}"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return f"Cannot open: {video_path}"

    img_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    img_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_annotations = {}
    video_record = ann_dict.get(video_id, {})
    
    for interval in video_record.get("annotations", []):
        for bbox_item in interval.get("bboxes", []):
            frame_num = bbox_item['frame']
            if frame_num not in frame_annotations:
                frame_annotations[frame_num] = []
            frame_annotations[frame_num].append(bbox_item)

    img_out = train_img_dir if mode == "train" else val_img_dir
    lbl_out = train_lbl_dir if mode == "train" else val_lbl_dir
    
    saved_obj = 0
    saved_bg = 0

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        should_save = False
        is_background = False

        if frame_idx in frame_annotations:
            should_save = True
            is_background = False
        
        elif frame_idx % EMPTY_FRAME_STEP == 0:
            should_save = True
            is_background = True
        
        if should_save:
            image_name_stem = f"{video_id}_frame_{frame_idx:06d}"
            img_path = os.path.join(img_out, image_name_stem + ".jpg")
            txt_path = os.path.join(lbl_out, image_name_stem + ".txt")

            success = cv2.imwrite(img_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            if not success:
                print(f"Failed to write: {img_path}")
                frame_idx += 1
                continue

            # Lưu Label
            if not is_background:
                yolo_labels = []
                for bbox in frame_annotations[frame_idx]:
                    yolo_line = convert_to_yolo_format(
                        bbox['x1'], bbox['y1'], bbox['x2'], bbox['y2'], 
                        img_width, img_height, 0
                    )
                    yolo_labels.append(yolo_line)
                
                with open(txt_path, "w") as f:
                    f.write('\n'.join(yolo_labels))
                saved_obj += 1
            else:
                # Ghi file rỗng (Background image)
                with open(txt_path, "w") as f:
                    pass 
                saved_bg += 1
        
        frame_idx += 1

    cap.release()
    return f"{video_id}: {saved_obj} annotated + {saved_bg} background frames saved."

print("Extracting frames + labels ...")
futures = []
with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    for vid in train_videos:
        futures.append(ex.submit(extract_frames_and_labels, vid, "train"))
    for vid in val_videos:
        futures.append(ex.submit(extract_frames_and_labels, vid, "val"))
    
    for i, fut in enumerate(as_completed(futures), 1):
        print(f"[{i}/{len(futures)}] {fut.result()}")

print("All videos processed!")

print("\nValidating extracted images...")
train_imgs = glob.glob(os.path.join(train_img_dir, "*.jpg"))
val_imgs = glob.glob(os.path.join(val_img_dir, "*.jpg"))
print(f"  Train images: {len(train_imgs)}")
print(f"  Val images: {len(val_imgs)}")

test_success = 0
for img_path in train_imgs[:5]:
    img = cv2.imread(img_path)
    if img is not None:
        test_success += 1
print(f" Test read: {test_success}/5 successful ")

data_yaml = {
    "train": os.path.abspath(train_img_dir),
    "val": os.path.abspath(val_img_dir),
    "nc": 1,                 
    "names": CLASS_NAMES
}
data_path = os.path.join(WORK_DIR, "data.yaml")
with open(data_path, "w") as f:
    yaml.dump(data_yaml, f)
print(f"data.yaml created for Single Class")

Extracting frames + labels ...
[1/14] WaterBottle_0: 934 annotated + 136 background frames saved.
[2/14] Lifering_0: 1134 annotated + 120 background frames saved.
[3/14] MobilePhone_0: 968 annotated + 181 background frames saved.
[4/14] Lifering_1: 1511 annotated + 174 background frames saved.
[5/14] Person1_1: 1129 annotated + 134 background frames saved.
[6/14] Laptop_1: 987 annotated + 139 background frames saved.
[7/14] Jacket_0: 1162 annotated + 133 background frames saved.
[8/14] MobilePhone_1: 889 annotated + 134 background frames saved.
[9/14] Jacket_1: 690 annotated + 152 background frames saved.
[10/14] WaterBottle_1: 3123 annotated + 122 background frames saved.
[11/14] Laptop_0: 884 annotated + 145 background frames saved.
[12/14] Backpack_1: 1454 annotated + 102 background frames saved.
[13/14] Person1_0: 2057 annotated + 118 background frames saved.
[14/14] Backpack_0: 3184 annotated + 243 background frames saved.
All videos processed!

Validating extracted images...
  Tr

In [11]:
model = YOLO(YOLO_MODEL)

In [12]:
# Train model
print("\nStarting YOLO training...")
results = model.train(
    data=data_path,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=640,
    workers=NUM_WORKERS,          
    exist_ok=True,      
    project="/kaggle/working/yolo_finetune",
    device=[0, 1],  # Thêm dòng này để sử dụng GPU 0 và 1
)
print("Training completed!")


Starting YOLO training...
Ultralytics 8.3.229 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/stream_dataset/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       7158       6695      0.855      0.618      0.713      0.405
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /kaggle/working/yolo_finetune/train
Training completed!


In [13]:
class YOLODetector:
    def __init__(self, model_path, confidence=0.0001):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = YOLO(model_path)
        # self.model.set_classes(['target'])
        self.conf = confidence

    def detect_video(self, video_path):
        """
        Chạy model YOLO trên video và trả về tất cả các detection tìm thấy.
        """
        if not os.path.exists(video_path):
            print(f"Warning: File {video_path} not found.")
            return []

        detections = []

        results = self.model.predict(
            source=video_path,
            conf=self.conf,
            iou=0.5,        # NMS IoU threshold của YOLO
            imgsz=640,
            stream=True,
            verbose=False
        )

        for frame_idx, r in enumerate(results):
            if r.boxes is not None and len(r.boxes) > 0:
                # Lấy tọa độ xyxy và chuyển sang numpy
                boxes = r.boxes.xyxy.cpu().numpy()
                
                # Lưu lại tất cả các box tìm thấy trong frame này
                for (x1, y1, x2, y2) in boxes:
                    detections.append({
                        "frame": frame_idx,
                        "x1": int(x1), 
                        "y1": int(y1),
                        "x2": int(x2), 
                        "y2": int(y2)
                    })

        print(f"-> Found {len(detections)} raw detections.")
        return detections

    def process_test_set(self, test_root, output_json):
        """
        Duyệt qua thư mục test, chạy detect và lưu vào JSON.
        """
        video_dirs = sorted([d for d in Path(test_root).iterdir() if d.is_dir()])
        submission = []
        
        print(f"Starting processing on {self.device}...")
        
        for i, vid_dir in enumerate(video_dirs, 1):
            video_path = vid_dir / "drone_video.mp4"
            print(f"\n[{i}/{len(video_dirs)}] 🎥 {vid_dir.name}")

            if 'images' in vid_dir.name:
                continue
                            
            # Lấy danh sách raw detections
            detections = self.detect_video(str(video_path))

            # Đóng gói theo format yêu cầu
            submission.append({
                "video_id": vid_dir.name,
                "detections": [{"bboxes": detections}] if detections else []
            })

        # Lưu file JSON
        with open(output_json, "w") as f:
            json.dump(submission, f, indent=2)

        print(f"\n✅ Process completed. Saved to: {output_json}")
        return submission

In [14]:
# Run inference
TRAINED_MODEL_PATH = 'yolo_finetune/train/weights/best.pt' 
detector = YOLODetector(TRAINED_MODEL_PATH, confidence=0.05)

submission = detector.process_test_set(
    test_root= TEST_ROOT,
    output_json="/kaggle/working/submission.json"
)
print("COMPLETE")

Starting processing on cuda...

[1/12] 🎥 BlackBox_0
-> Found 2072 raw detections.

[2/12] 🎥 BlackBox_0_images

[3/12] 🎥 BlackBox_1
-> Found 1284 raw detections.

[4/12] 🎥 BlackBox_1_images

[5/12] 🎥 CardboardBox_0
-> Found 425 raw detections.

[6/12] 🎥 CardboardBox_0_images

[7/12] 🎥 CardboardBox_1
-> Found 314 raw detections.

[8/12] 🎥 CardboardBox_1_images

[9/12] 🎥 LifeJacket_0
-> Found 7746 raw detections.

[10/12] 🎥 LifeJacket_0_images

[11/12] 🎥 LifeJacket_1
-> Found 4404 raw detections.

[12/12] 🎥 LifeJacket_1_images

✅ Process completed. Saved to: /kaggle/working/submission.json
COMPLETE
